# Diseases notebook

The goal of this notebook is to estimate odds about disease apperance for each cause from SDF logs.


First, let's import libraries and the logs :

In [1]:
import pandas as pd
import utils

In [2]:
logs = utils.get_player_logs()

In [3]:
logs.head()

,Ship,Character,Day.Cycle,Action,Log,Day,Cycle
0,1,Janice,2.4,NEW_CREW_MEMBER,*Janice* s'est éveillée de son si long sommeil.,2.0,4.0
1,1,Janice,2.4,CHARACTER_LEFT,*Janice* est sortie.,2.0,4.0
2,1,Janice,2.4,CHARACTER_ENTERED,*Janice* est entrée.,2.0,4.0
3,1,Janice,2.4,CUDDLE_OTHER,"*Janice* réconforte *Finola*, ça ira mieux de...",2.0,4.0
4,1,Janice,2.4,CHARACTER_LEFT,*Janice* est sortie.,2.0,4.0


## Trauma
Players can catch diseases and disorders by witnessing a death. We will call this event a **Trauma**.
We want to estimate :
- the odds of getting a trauma
- the mass function of the diseases/disorders (if the trauma event is drawn, what are the odds of getting a specific disease ?)

First, let's extract all the `fist kills` logs (a subset of death logs) :

In [4]:
utils.find_all_actions_by_name(logs, "FIST_KILLED")

,Ship,Character,Day.Cycle,Action,Log,Day,Cycle
265,1,Janice,6.5,FIST_KILLED,*Janice* s'acharne sur *Eleesha*...qui rend s...,6.0,5.0
2177,2,Chao,4.5,FIST_KILLED,*Chao* s'acharne sur *Paola*...qui rend son d...,4.0,5.0
2209,2,Chao,4.7,FIST_KILLED,*Chao* s'acharne sur *Janice*...qui rend son ...,4.0,7.0
2810,2,Frieda,4.1,FIST_KILLED,*Frieda* s'acharne sur *Eleesha*...qui rend s...,4.0,1.0
3496,2,Kuan_Ti,6.6,FIST_KILLED,*Kuan Ti* s'acharne sur *Raluca*...qui rend s...,6.0,6.0
...,...,...,...,...,...,...,...
7723279,1707,Paola,1.8,FIST_KILLED,*Paola* s'acharne sur *Eleesha*...qui rend so...,1.0,8.0
7723404,1707,Ian,1.8,FIST_KILLED,*Ian* s'acharne sur *Chun*...qui rend son der...,1.0,8.0
7724967,1709,Kuan_Ti,3.5,FIST_KILLED,*Kuan Ti* s'acharne sur *Paola*...qui rend so...,3.0,5.0
7725273,1709,Janice,4.5,FIST_KILLED,*Janice* s'acharne sur *Terrence*...qui rend ...,4.0,5.0


Let's see if we can find the `Trauma` logs before or after the kill logs :

In [8]:
logs.loc[2808:2812]

,Ship,Character,Day.Cycle,Action,Log,Day,Cycle
2808,2,Frieda,4.1,TRAUMA_DISEASE,Ce que vous venez de voir vous a choqué. Vous...,4.0,1.0
2809,2,Frieda,4.1,DISEASED_PSY,Vous ne vous sentez pas très très bien... Vot...,4.0,1.0
2810,2,Frieda,4.1,FIST_KILLED,*Frieda* s'acharne sur *Eleesha*...qui rend s...,4.0,1.0
2811,2,Frieda,4.1,CHARACTER_LEFT,*Frieda* est sortie.,4.0,1.0
2812,2,Frieda,4.1,CHARACTER_ENTERED,*Frieda* est entrée.,4.0,1.0


We see on this first example that the `TRAUMA_DISEASE` log is 2 logs before the kill log, and the `DISEASED` event log is the log just before.

In [10]:
logs.loc[263:267]

,Ship,Character,Day.Cycle,Action,Log,Day,Cycle
263,1,Janice,6.5,FIST_WOUNDED,*Janice* met une terrible baigne à *Eleesha*...,6.0,5.0
264,1,Janice,6.5,TRAUMA_DISEASE,Ce que vous venez de voir vous a choqué. Vous...,6.0,5.0
265,1,Janice,6.5,FIST_KILLED,*Janice* s'acharne sur *Eleesha*...qui rend s...,6.0,5.0
266,1,Janice,6.5,TRIUMPH_EARNED,Vous avez gagné *3 Triomphe*.,6.0,5.0
267,1,Janice,6.6,DMG_DEALT,Vous perdez 3 hp.,6.0,6.0


This second example is slightly different : we have a Trauma log but not a disease log. 
 
It's probably because Janice is a Mush : she should have catched the disease but is immunized.

In [11]:
logs.loc[2175:2179]

,Ship,Character,Day.Cycle,Action,Log,Day,Cycle
2175,2,Chao,4.5,FIST_WOUNDED,*Chao* mets un magistral uppercut à *Paola*...,4.0,5.0
2176,2,Chao,4.5,MORAL_DOWN,Vous avez perdu 1 moral.,4.0,5.0
2177,2,Chao,4.5,FIST_KILLED,*Chao* s'acharne sur *Paola*...qui rend son d...,4.0,5.0
2178,2,Chao,4.5,DIRTED,C'est dégoûtant... Vous vous êtes sali.,4.0,5.0
2179,2,Chao,4.6,LOG_ACCESS,*Chao* a accédé au *Centre de Communication*.,4.0,6.0


We can see on the third example that if a human player doesn't catch a disease after a kill, the log just before is a `MORAL_DOWN` event.

In [4]:
logs.loc[7723402:7723406]

,Ship,Character,Day.Cycle,Action,Log,Day,Cycle
7723402,1707,Ian,1.8,FIST_MISSED,*Ian* rate son coup sur *Chun*.,1.0,8.0
7723403,1707,Ian,1.8,FIST_WOUNDED,*Ian* met une terrible baigne à *Chun*...,1.0,8.0
7723404,1707,Ian,1.8,FIST_KILLED,*Ian* s'acharne sur *Chun*...qui rend son der...,1.0,8.0
7723405,1707,Ian,1.8,TRIUMPH_EARNED,Vous avez gagné *3 Triomphe*.,1.0,8.0
7723406,1707,Ian,1.8,DMG_DEALT,Vous perdez 3 hp.,1.0,8.0


For a Mush, the log before the kill one is a `FIST_WOUNDED` log (if they doesn't trigger a `TRAUMA_DISEASE` event).

In [5]:
logs.loc[300728:300732]

,Ship,Character,Day.Cycle,Action,Log,Day,Cycle
300728,69,Jin_Su,4.8,FIST_WOUNDED,*Jin Su* mets un magistral uppercut à *Janice...,4.0,8.0
300729,69,Jin_Su,4.8,MORAL_DOWN,Vous avez perdu 1 moral.,4.0,8.0
300730,69,Jin_Su,4.8,SKILL_ADD_PA,Votre compétence *Sang-froid* a porté ses fru...,4.0,8.0
300731,69,Jin_Su,4.8,FIST_KILLED,*Jin Su* s'acharne sur *Janice*...qui rend so...,4.0,8.0
300732,69,Jin_Su,4.8,TRIUMPH_EARNED,Vous avez gagné *3 Triomphe*.,4.0,8.0


Another edge case to keep in mind is the Cold-blooded skill which prints a log before the kill one.

Let's recap by taking all the logs just before the kill ones :

In [8]:
potential_traumas = logs.loc[utils.find_all_actions_by_name(logs, "FIST_KILLED").index - 1]
potential_traumas

,Ship,Character,Day.Cycle,Action,Log,Day,Cycle
264,1,Janice,6.5,TRAUMA_DISEASE,Ce que vous venez de voir vous a choqué. Vous...,6.0,5.0
2176,2,Chao,4.5,MORAL_DOWN,Vous avez perdu 1 moral.,4.0,5.0
2208,2,Chao,4.7,MORAL_DOWN,Vous avez perdu 1 moral.,4.0,7.0
2809,2,Frieda,4.1,DISEASED_PSY,Vous ne vous sentez pas très très bien... Vot...,4.0,1.0
3495,2,Kuan_Ti,6.6,MORAL_DOWN,Vous avez perdu 1 moral.,6.0,6.0
...,...,...,...,...,...,...,...
7723278,1707,Paola,1.8,DISEASED,Vous ne vous sentez pas très très bien... Cha...,1.0,8.0
7723403,1707,Ian,1.8,FIST_WOUNDED,*Ian* met une terrible baigne à *Chun*...,1.0,8.0
7724966,1709,Kuan_Ti,3.5,MORAL_DOWN,Vous avez perdu 1 moral.,3.0,5.0
7725272,1709,Janice,4.5,MORAL_DOWN,Vous avez perdu 1 moral.,4.0,5.0


In [13]:
events_repartition = potential_traumas.value_counts("Action")
events_repartition

Action
MORAL_DOWN        2336
FIST_WOUNDED       805
DISEASED           628
DISEASED_PSY       604
TRAUMA_DISEASE     268
SKILL_ADD_PA        61
dtype: int64

We can see we've browsed all the cases.

As said above, Mush players can get the `TRAUMA_DISEASE` (trauma event triggered) or the `FIST_WOUNDED` log before the kill log.
 
Human players can get the `DISEASED` or `DISEASE_PSY` log if they triggered the trauma event or the `MORAL_DOWN` log otherwise.

Then if we take only human logs, an estimation of the `Trauma` event probability is $\frac{\#diseased + \#diseased psy}{\#diseased + \#diseased psy + \#moral down}$.

In [29]:
diseased_logs = events_repartition["DISEASED"] + events_repartition["DISEASED_PSY"]
moral_down_logs = events_repartition["MORAL_DOWN"]

trauma_probability_1 = diseased_logs / (diseased_logs + moral_down_logs)

In [30]:
print("First estimation of Trauma event probability : {:.2f}%".format(trauma_probability_1 * 100))

First estimation of Trauma event probability : 34.53%


With similar reasoning we can estimate this probability with kills done with a knife :

In [24]:
knife_potential_traumas = logs.loc[utils.find_all_actions_by_name(logs, "KNIFE_KILLED").index - 1]
knife_event_repartition = knife_potential_traumas.value_counts("Action")

In [26]:
knife_event_repartition

Action
MORAL_DOWN        731
KNIFE_WOUNDED     535
DISEASED          232
DISEASED_PSY      176
TRAUMA_DISEASE    113
SKILL_ADD_PA       33
dtype: int64

In [32]:
diseased_logs = knife_event_repartition["DISEASED"] + knife_event_repartition["DISEASED_PSY"]
moral_down_logs = knife_event_repartition["MORAL_DOWN"]

trauma_probability_2 = diseased_logs / (diseased_logs + moral_down_logs)

In [33]:
print("2nd estimation of Trauma event probability : {:.2f}%".format(trauma_probability_2 * 100))

2nd estimation of Trauma event probability : 35.82%
